## Aufgabe 1

...

## Aufgabe 2

In [1]:
import glob
import os.path
import email

import pandas as pd

In [2]:
def rent_in_rome(n=None, cols=['Title', 'Short Description']):
    return pd.read_csv('data/dataset_rent_rome_kijiji.tsv', sep='\t', header=0,
                       nrows=n, usecols=cols)


rent_in_rome().head()

,Title,Short Description
0,Studio accessoriato vicino metro A Furio Camillo,Affitto studio a professionisti preferibilment...
1,"Negozio 169Mq per laboratorio, ufficio, studio...","Privato affitta negozio 169 mq, al piano terra..."
2,Negozio in tiburtina centro,Negozio c/1 roma tiburtina centro via eugenio ...
3,Studio medico via anapo parco nemorense,"Studio medico avviato, composto da tre studi c..."
4,Cerco: Appartamento per donna lavoratrice refe...,"Donna lavoratrice, non residente, con reddito ..."


In [3]:
def read_file(path):
    with open(path, 'r', encoding='utf-8', errors='ignore') as file:
        raw_email = file.read()
        msg = email.message_from_string(raw_email)
        return {
            'file': path,
            'subject': msg['subject'],
            'from': msg['from'],
            'to': msg['to'],
            'body': msg.get_payload(decode=True).decode(errors='ignore')
        }


def enron_email(n=None):
    paths = glob.glob(f"data/maildir/**/*", recursive=True)[:n]
    return pd.DataFrame(read_file(path) for path in paths if os.path.isfile(path))


enron_email(5).head()

""


In [4]:
rir_data = rent_in_rome()
ee_data = enron_email()

## Aufgabe 3

In [5]:
class KShingles:
    def __init__(self, k):
        assert k > 0
        self.k = k

    def __call__(self, entry):
        assert len(entry) <= self.k
        for i in range(len(entry) - self.k + 1):
            yield entry[i:i + self.k]

Ein paar assertions sollten reichen

## Aufgabe 4

In [6]:
from k_shingles import MR4Shingles
import tempfile

In [7]:
def run(input_file, output_file):
    job = MR4Shingles(args=[input_file])
    with job.make_runner() as runner, open(output_file, 'w') as out:
        runner.run()
        out.writelines(f"{key}\t{value}\n" for key, value in
                       sorted(job.parse_output(runner.cat_output()), key=lambda k: -k[1])[:20])

In [8]:
with tempfile.NamedTemporaryFile('w+') as tmp:
    tmp.writelines(rir_data['Title'] + rir_data['Short Description'])
    run(tmp.name, './rir')

No configs specified for inline runner


In [ ]:
with tempfile.NamedTemporaryFile('w+') as tmp:
    tmp.writelines(ee_data['body'])
    run(tmp.name, './ee')

No configs specified for inline runner


## Aufgabe 5
1. Texte auswählen (z.B. Title + Description oder nur Description)
2. Text in Dokumente (set of shingles) transformieren
3. Nummerierte Liste von unique Shingles [a,b,...]
4. Dokumente in Vektor Form transformieren (Index aus nummerierter Shingle Liste)
5. H Hash Funktionen generieren
6. Vektor Form Dokumente in Signaturen transformieren